# 경사하강(기울기 계산)

자동 경사하강 계산 (오토가드) 패키지를 이용하여 기울기를 계산하는 법을 배웁니다. 기울기 계산은 모델 최적화에 필수적이기에 이 부분은 우리가 이해해야할 매우 중요한 주제입니다.
하지만 파이토치의 오토가드 패키지는 계산에 필요한 모든 모듈을 제공하기에 우리가 어떻게 사용해야하는지만 알면 됩니다.

In [2]:
import torch

x = torch.randn(3, requires_grad=True)
print(x)

y = x+2
print(y)

z = y*y*2
print(z)


tensor([ 1.2630, -1.5445, -2.4180], requires_grad=True)
tensor([ 3.2630,  0.4555, -0.4180], grad_fn=<AddBackward0>)
tensor([21.2941,  0.4150,  0.3494], grad_fn=<MulBackward0>)


위의 코드에서 `requires_grad`를 `True`로 설정하면 파이토치는 그 텐서의 모든 연산에 대해 `computational graph`를 생성합니다.

이는 다음과 같이 생겼습니다. (하단의 이미지란 참고)

각각의 연산에 대해서 파이토치는 인풋과 아웃풋이 있는 노드를 생성합니다.

여기서 연산은 "더하기" 이며 인풋은 `x`와 `2`이며 아웃풋은 `y`입니다.

이 그래프에 대해서 `back propagation(오차 역전파)` 기술을 이용해 경사를 계산합니다.

> `back propagation (오차 역전파)`에 대한 자세한 내용은 다음 튜토리얼에서 설명합니다.

맨 처음 `Forward`(이미지 참고)로 계산이 이루어집니다. 

이 과정에서 y가 계산되며 우리가 `x`에 대해 `requires_grad`를 `True`로 지정해 두었기 때문에 

파이토치는 자동으로 함수를 생성 후 저장합니다.

> 여기서 함수는 `x + 2 = y`입니다.

그리고 여기서 함수는 `back propagation (오차 역전파)`를 위해 경사를 계산하는데 사용됩니다.

`y`의 `grad_fn`속성이 그라디언트 함수를 가리킬것입니다.

이 경우 `Backward`로 호출되기에 역방향 경로에서 기울기를 계산합니다.

이 경우 x에 대한 y의 기울기를 계산합니다.

## backward

`backward`는 이미지에서 보시다시피 지금까지 한 연산의 기울기(미분)을 하는 것입니다. 다음 기울기 연산이 진행될때 `벡터 자코비안` (필자는 수학에 대해 자세히 모릅니다 ㅠㅠ 자세한 사항은 [여기](https://angeloyeo.github.io/2020/07/24/Jacobian.html)를 참고해주세요)이 이용됩니다.

`backward` 메소드는 텐서가 스칼라 값(단일 값) 일 경우 혹은 `backward`안에 벡터를 넣어 주어야 정상적으로 작동합니다. 그렇지 않을 경우 에러를 리턴합니다.

In [3]:

z.backward() # dz/dx
print(x.grad)

RuntimeError: grad can be implicitly created only for scalar outputs

In [4]:
v = torch.tensor([0.1, 1.0, 0.01], dtype=torch.float32)
z.backward(v)
print(x.grad)

tensor([ 1.3052,  1.8220, -0.0167])


## grad_fn에 포함시키지 않고 연산하기

`requires_grad`가 `True`인 텐서의 모든 연산은 `grad_fn`에 포함됩니다. 하지만 우린 그 함수에 포함시키지 않고 값을 변경시키고 싶을때가 있습니다. (주로 훈련중에) 다은 3가지 방법은 `grad_fn`에 포함시키지 않고 텐서의 값을 변경시킬수 있게 합니다.

In [5]:
x.requires_grad_(False) # x의 requires_grad를 False로 만듭니다.

y = x.detach() # requires_grad가 False인 텐서로 복사합니다.

with torch.no_grad(): # 다음 with가 있는 동안은 grad_fn에 포함시키지 않습니다.
  pass 

tensor([ 1.2630, -1.5445, -2.4180])

## 주의 사항 - 기울기의 합

`backward` 메소드를 실행시키게 되면 기울기는 `x.grad`에 저장됩니다. 하지만 `x.grad`에 값이 저장되어 있는 상태로 `backward`를 실행시킬 경우 그 `x.grad`에 덮어쒸워 지는게 아닌 합연산이 이루어지게 됩니다. 

In [ ]:
weights = torch.ones(4, requires_grad=True)

for epoch in range(3):
  model_output = weights * 3

  model_output.backward()

  print(weights.grad)

# 이미지

![img](img/autograd.png)